In [1]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense #Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout, SimpleRNN
from keras.utils import plot_model
from netvladlayer import NetVLAD
from keras import Sequential

In [2]:
def define_model(vocab_size, max_length,embedding_dim,hidden_size):
  # features from the CNN model compressed from 2048 to 256 nodes
  # inputs1 = Input(shape=(2048,))
  img_input = Input(shape=(224, 224, 3))
  cnn_model = load_model('qn2_cnn.keras')
  cnn_model = Model(inputs=cnn_model.inputs, outputs=cnn_model.layers[-3].output)
  cnn_model.trainable = False
  cnn_output = cnn_model(img_input)

  # net_vlad = NetVLAD(num_clusters=64, output_dim=512)
  # vlad_layer = net_vlad(cnn_output)
  # fe1 = Dropout(0.5)(cnn_output)

  fe2 = Dense(hidden_size, activation='relu')(cnn_output)  
  
  # RNN layer
  inputs2 = Input(shape=(max_length,))
  se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
  se1.trainable = False
  #set the embedding matrix as the weight of the embedding layer###########

  se3 = SimpleRNN(hidden_size, return_sequences=True)(se1, initial_state=[fe2])
  decoder2 = Dense(256, activation='relu')(se3)
  outputs = Dense(vocab_size, activation='softmax')(decoder2)
  
  model = Model(inputs=[img_input, inputs2], outputs=outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  # summarize model
  print(model.summary())
  return model

In [3]:
model = define_model(400000, 40,300,256)
# model.layers[2].set_weights([embedding_matrix])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 40)]                 0         []                            
                                                                                                  
 model (Functional)          (None, 20412)                371       ['input_1[0][0]']             
                                                                                                  
 embedding (Embedding)       (None, 40, 300)              1200000   ['input_2[0][0]']             
                                                          00                                

In [1]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = np.array([[1, 2, 3],[2, 3, 4],[3, 4, 5], [4, 5, 6]])
y = np.array([1, 2, 3, 4])

reg = LinearRegression()
reg.fit(X, y)
print("Coefficients:", reg.coef_)
print("Intercept:", reg.intercept_)
y_pred = reg.predict(X)
print("Predictions:", y_pred)


from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)




Coefficients: [0.33333333 0.33333333 0.33333333]
Intercept: -1.0000000000000004
Predictions: [1. 2. 3. 4.]
Mean Squared Error: 1.9721522630525295e-31
R^2 Score: 1.0


In [2]:
# BLEU evaluation
from nltk.translate.bleu_score import corpus_bleu

def evaluate_bleu(model, data, eng_tokenizer, bert_tokenizer):
  references = []
  candidates = []

  for i, row in data.iterrows():
      input_seq = tokenize_and_pad([row['source']], eng_tokenizer, max_eng_len)
      decoded_sentence = decode_sequence(input_seq)
      references.append([row['source'].split()])
      candidates.append(decoded_sentence.split())

  bleu_scores = {
      'BLEU-1': corpus_bleu(references, candidates, weights=(1, 0, 0, 0)),
      'BLEU-2': corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)),
      'BLEU-3': corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33, 0)),
      'BLEU-4': corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25))
  }

  return bleu_scores

# Evaluate on test data
test_bleu_scores = evaluate_bleu(model, test_data, eng_tokenizer, bert_tokenizer)
print(f"Test BLEU Scores: {test_bleu_scores}")


[5.]
